In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

FRONTIER_COUNT=6
FRONTIER_FEATURES=6
ACTION_SPACE=FRONTIER_COUNT 
OTHER_FRONTIER_INPUTS=3

In [2]:
Q1 = torch.load("data/Q_function_14.pth", map_location=torch.device('cpu'), weights_only=False)
Q2 = torch.load("data/Q_function_13.pth", map_location=torch.device('cpu'), weights_only=False)

In [3]:
keys = [
    'size',
    'count',
    'distance',
    'repulsion_angle',
    'direction_angle',
    'distance_last'
]
keys.sort()
keys

global_keys = [
    'total_exploration',
    'exploration progress',
    'local_exploration'
]

In [4]:
def multi_input1(x):
    output = torch.zeros(ACTION_SPACE,device=x.device, dtype=x.dtype)
    for i in range(ACTION_SPACE):
        input = torch.zeros(FRONTIER_FEATURES+OTHER_FRONTIER_INPUTS, device=x.device, dtype=x.dtype)
        for j in range(ACTION_SPACE):
            input[j] = x[j*FRONTIER_FEATURES+i]
        for j in range(OTHER_FRONTIER_INPUTS):
            input[FRONTIER_FEATURES+j] = x[ACTION_SPACE*FRONTIER_FEATURES+j]
        output[i] = Q1(input)[0]
    return output

def multi_input2(x):
    output = torch.zeros(ACTION_SPACE,device=x.device, dtype=x.dtype)
    for i in range(ACTION_SPACE):
        input = torch.zeros(FRONTIER_FEATURES+OTHER_FRONTIER_INPUTS, device=x.device, dtype=x.dtype)
        for j in range(ACTION_SPACE):
            input[j] = x[j*FRONTIER_FEATURES+i]
        for j in range(OTHER_FRONTIER_INPUTS):
            input[FRONTIER_FEATURES+j] = x[ACTION_SPACE*FRONTIER_FEATURES+j]
        output[i] = Q2(input)[0]
    return output

policy1 = multi_input1
policy2 = multi_input2

In [5]:
actions = np.zeros(6)
for _ in range(1000):
    obs = torch.tensor(np.random.random(39), dtype=torch.float32)
    obs[-3] = 0.5
    obs[-2] = 0.01
    obs[-1] = 0.2
    with torch.no_grad():
        out = policy1(obs).cpu().numpy()
        actions[np.argmax(out)] += 1
print(actions)

[158. 159. 147. 182. 168. 186.]


In [9]:
weights1 = Q1[0][0].weight.detach().numpy()[0,:]
bias1 = Q1[0][0].bias.detach().numpy()[0]
weights2 = Q2[0][0].weight.detach().numpy()[0,:]
bias2 = Q2[0][0].bias.detach().numpy()[0]
X = np.linspace(0,1,100)

In [11]:
for i in range(FRONTIER_FEATURES):
    print(f"{keys[i]}: {weights1[i]:.2f}  / {weights2[i]:.2f}")
for i in range(OTHER_FRONTIER_INPUTS):
    print(f"{global_keys[i]}: {weights1[i+FRONTIER_FEATURES]:.2f} / {weights2[i+FRONTIER_FEATURES]:.2f}")


count: 0.93  / 1.06
direction_angle: 1.11  / 1.18
distance: 1.08  / 1.05
distance_last: 1.25  / 1.16
repulsion_angle: 0.52  / 0.39
size: 0.79  / 0.88
total_exploration: 1.13 / 1.25
exploration progress: 0.66 / 0.71
local_exploration: 0.03 / 0.04
